In [597]:
import numpy as np
from scipy.optimize import minimize
import powerlaw
import matplotlib.pyplot as plt
plt.style.use('seaborn-paper')
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 150

/var/folders/77/jfwlc7mn17nftm69gps4bk640000gn/T/ipykernel_24350/2083958521.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-paper')


In [598]:
# establish the dataset
d = 50 # dimension
k = 3 # sparsity
n = 120 # sample size
K = 40 # number of subgroups
epsilon = 0.05 # corruption probability
mu = np.zeros(d)
mu_sparse = [100,50,20]
mu[:k] = mu_sparse
mu = mu.reshape((d, 1))

setting = 'normal'
distribution = 'powerlaw'
X = np.empty((n, d))

match distribution:
    case 'normal':
        # normal distribution
        X = np.random.normal(0, 1, size=(n, d))
    case 'powerlaw':
        # power law distribution
        alpha = 2.5
        xmin = 1
        mean = np.zeros((n, d))
        mean += (alpha / (alpha - 1)) * xmin
        dist = powerlaw.Power_Law(xmin=xmin, parameters=[alpha])
        for i in range(n):
            X[i, :] = dist.generate_random(d)
        X = X - mean


for i in range(k):
    X[:, i] += mu_sparse[i]

# print(np.mean(X, axis=0))

# Add corrupted data
# for j in range(np.floor(epsilon * n).astype(int)):
#     X[j, :] = 50 * np.random.standard_cauchy(size=(1, d)) # non-standard Cauchy, mean 20, scale 50

# Grouping preprocessing
X_grouped = np.split(X, K)
X_grouped = np.mean(X_grouped, axis=1)

# print(len(X_grouped))

In [599]:
def robust_sparse_mean_estimation(X, k, epsilon):
    """
    Computes a robust sparse estimate of the mean of a distribution given an epsilon-corrupted set of samples.

    Parameters:
        X (numpy array): a 2D array of shape (d, n) representing the n samples from a d-dimensional distribution
        k (int): the sparsity parameter, i.e. the number of coordinates to include in the estimate
        epsilon (float): the corruption parameter, i.e. the maximum fraction of corrupted samples

    Returns:
        mu_hat (numpy array): a 1D array of shape (d,) representing the estimated mean
    """
    d, n = X.shape

    # Define the objective function
    def f(w):
        Sigma_w = np.dot(X, np.dot(np.diag(w), X.T)) - np.outer(mu_w, mu_w)
        V = np.linalg.eigh(Sigma_w)[1][:, -k:]
        Lambda = np.diag(np.linalg.eigh(Sigma_w)[0][-k:])
        return np.linalg.norm(np.dot(np.dot(V, Lambda), V.T) - np.eye(d), 'fro')**2

    # Define the gradient of the objective function
    def grad_f(w):
        Sigma_w = np.dot(X, np.dot(np.diag(w), X.T)) - np.outer(mu_w, mu_w)
        V = np.linalg.eigh(Sigma_w)[1][:, -k:]
        Lambda = np.diag(np.linalg.eigh(Sigma_w)[0][-k:])
        M = np.dot(np.dot(V, Lambda), V.T) - np.eye(d)
        G = np.zeros(n)
        for i in range(n):
            x = X[:, i]
            G[i] = np.dot(np.dot(x - mu_w, V), np.dot(np.dot(Lambda, V.T), x - mu_w))
        return 4 * np.dot(np.dot(X, G), X - np.outer(mu_w, np.ones(n))) / n**2

    # Perform gradient descent to minimize f(w)
    mu_w = np.zeros(d)
    w = np.ones(n) / n
    for i in range(1000):

        print(w)
        print(grad_f(w))
        w_new = w - 0.1 * grad_f(w)
        w_new = np.maximum(w_new, 0)
        w_new /= np.sum(w_new)
        if np.linalg.norm(w_new - w) < 1e-8:
            break
        w = w_new
        mu_w = np.dot(X, w)
    # Compute the estimated mean
    mu_hat = np.dot(X, w)

    # Return the top k coordinates of mu_hat with largest magnitude
    Q = np.argsort(np.abs(mu_hat))[::-1][:k]
    return mu_hat[Q]


In [600]:

# def robust_sparse_mean_estimation(X, k, epsilon):
#     n = X.shape[0]
#     D = X.shape[1]
    
#     def obj_func(w):
#         S = np.dot(X.T, w)
#         Sigma_w = np.dot(S[:, np.newaxis], S[np.newaxis, :]) / n
#         return np.linalg.norm(Sigma_w - np.eye(D), 'fro')**2
    
#     def constraint(w):
#         return np.sum(np.abs(w)) - k
    
#     cons = {'type': 'ineq', 'fun': constraint}
#     bounds = [(0, 1) for i in range(n)]
    
#     w0 = np.ones(n) / n
#     res = minimize(obj_func, w0, bounds=bounds, constraints=cons, method='SLSQP')
#     w_hat = res.x
    
#     mu_w = np.dot(X.T, w_hat)
#     Q = np.abs(mu_w).argsort()[-k:]
#     mu_hat = np.zeros(D)
#     mu_hat[Q] = mu_w[Q]
    
#     return mu_hat

In [601]:
mu_hat = robust_sparse_mean_estimation(X_grouped, k, epsilon)
top_k_indices = np.abs(mu_hat).argsort()[-k:]
for i in top_k_indices:
    print("Coordinate {}: {}".format(i, mu_hat[i]))

/var/folders/77/jfwlc7mn17nftm69gps4bk640000gn/T/ipykernel_24350/1363202159.py:43: RuntimeWarning: invalid value encountered in divide
  w_new /= np.sum(w_new)


[0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
[3.19724529e+12 1.59016194e+12 6.59308520e+11 3.68012456e+10
 1.37571871e+10 3.90125408e+10 3.54289001e+10 3.34022648e+10
 3.61430500e+10 5.46545459e+10 1.90489624e+10 2.88138041e+10
 3.25614859e+10 2.75232916e+10 4.54455327e+10 8.76599488e+10
 7.63719457e+09 1.64210439e+11 3.51036381e+10 3.24524205e+10
 2.22393548e+10 4.70957347e+10 2.00085396e+10 3.50495687e+10
 3.75006164e+10 3.23249269e+10 3.81860561e+10 2.28898142e+10
 5.54025105e+10 2.51715372e+10 5.28480071e+10 4.24544793e+10
 5.42746871e+10 2.17857410e+10 3.21781122e+10 1.93876849e+10
 3.11354620e+10 2.26559677e+10 3.48387885e+10 3.12792250e+10
 1.97528780e+10 1.05116849e+11 1.10672737e+11 4.46059149e+10
 3.58473499e+10 3.41317086e+10 3.14562745e+10 1.61245625e+10
 2.88632158e+

Can't tolerate corruption even if epsilon = 0.05, standard Cauchy, covariance 50.